In [1]:
from bs4 import BeautifulSoup
import requests
import logging
import jaconv
from gensim import corpora
from gensim import models
from gensim.models import word2vec
from janome.tokenizer import Tokenizer
from pprint import pprint
import pandas as pd, numpy as np

In [2]:
url = "https://www.aozora.gr.jp/access_ranking/2019_xhtml.html"
URL = ""
res = requests.get(url)
res.encoding = 'shift-jis'
soup = BeautifulSoup(res.content, "html.parser")

In [3]:
url_list = [url["href"] for i, url in enumerate(soup.find_all("a", target="_blank")) if i < 50]

In [4]:
title = []
name = []
text = []
for url in url_list:
    res = requests.get(url)
    url_start = url[:37]
    res.encoding = 'shift-jis'
    soup = BeautifulSoup(res.content, "html.parser")
    for i, a in enumerate(soup.find_all("a")):
        if i == 7:
            url_end = a["href"][1:]
    url = url_start + url_end
    res = requests.get(url)
    res.encoding = 'shift-jis'
    soup = BeautifulSoup(res.content, "html.parser")
    title.append(soup.find("h1").string)
    name.append(soup.find("h2").string)
    for tag in soup.find_all(["rt", "rp"]):
        tag.decompose()
    text.append(soup.find("div",{'class': 'main_text'}).get_text())

In [5]:
df = pd.DataFrame({'タイトル': title, '作者': name, '本文': text})

In [6]:
df.head()

,タイトル,作者,本文
0,〔雨ニモマケズ〕,宮澤賢治,\r\n雨ニモマケズ\r\n風ニモマケズ\r\n雪ニモ夏ノ暑サニモマケヌ\r\n丈夫ナカラダ...
1,走れメロス,太宰治,\r\n メロスは激怒した。必ず、かの邪智暴虐の王を除かなければならぬと決意した。メロスには...
2,山月記,中島敦,\r\n 隴西の李徴は博学才穎、天宝の末年、若くして名を虎榜に連ね、ついで江南尉に補せられた...
3,こころ,夏目漱石,\n上 先生と私\n\n\n一\n\r\n 私はその人を常に先生と呼んでいた。だからここでも...
4,羅生門,芥川龍之介,\r\n ある日の暮方の事である。一人の下人が、羅生門の下で雨やみを待っていた。\r\n 広...


In [7]:
t = Tokenizer()
wakati_list = []
for i, doc in enumerate(text):
    print(i)
    if i == 0:
        doc = jaconv.kata2hira(doc)
    tokens = t.tokenize(doc)
    wakati = []
    for token in tokens:
        if token.part_of_speech.split(',')[0] not in ['記号']:
            wakati.append(token.surface)
    wakati_list.append(wakati)
#     test
    if i > 1:
        break

0
1
2


In [58]:
wakati_list[2]

['隴西',
 'の',
 '李',
 '徴',
 'は',
 '博学',
 '才',
 '穎',
 '天宝',
 'の',
 '末',
 '年',
 '若くして',
 '名',
 'を',
 '虎',
 '榜',
 'に',
 '連ね',
 'ついで',
 '江南',
 '尉',
 'に',
 '補せ',
 'られ',
 'た',
 'が',
 '性',
 '狷介',
 '自ら',
 '恃',
 'むところ',
 '頗る',
 '厚く',
 '賤吏',
 'に',
 '甘んずる',
 'を',
 '潔し',
 'と',
 'し',
 'なかっ',
 'た',
 'いくばく',
 'も',
 'なく',
 '官',
 'を',
 '退い',
 'た',
 '後',
 'は',
 '故山',
 '略',
 'に',
 '帰臥',
 'し',
 '人',
 'と',
 '交',
 'を',
 '絶っ',
 'て',
 'ひたすら',
 '詩作',
 'に',
 '耽っ',
 'た',
 '下',
 '吏',
 'と',
 'なっ',
 'て',
 '長く',
 '膝',
 'を',
 '俗悪',
 'な',
 '大官',
 'の',
 '前',
 'に',
 '屈する',
 'より',
 'は',
 '詩家',
 'として',
 'の',
 '名',
 'を',
 '死後',
 '百',
 '年',
 'に',
 '遺そ',
 'う',
 'と',
 'し',
 'た',
 'の',
 'で',
 'ある',
 'しかし',
 '文名',
 'は',
 '容易',
 'に',
 '揚ら',
 'ず',
 '生活',
 'は',
 '日',
 'を',
 '逐う',
 'て',
 '苦しく',
 'なる',
 '李',
 '徴',
 'は',
 '漸く',
 '焦躁',
 'に',
 '駆ら',
 'れ',
 'て',
 '来',
 'た',
 'この',
 '頃',
 'から',
 'その',
 '容貌',
 'も',
 '峭刻',
 'と',
 'なり',
 '肉',
 '落ち',
 '骨',
 '秀で',
 '眼光',
 'のみ',
 '徒',
 'ら',
 'に',
 '炯々',
 'として',
 '曾',
 'て',
 '進士',
 'に',
 '登',
 '第

In [69]:
model = word2vec.Word2Vec(wakati_list, size=200, min_count=1, window=10)

2020-10-21 07:57:31,889 : INFO : collecting all words and their counts
2020-10-21 07:57:31,895 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-10-21 07:57:31,905 : INFO : collected 2203 word types from a corpus of 8970 raw words and 3 sentences
2020-10-21 07:57:31,907 : INFO : Loading a fresh vocabulary
2020-10-21 07:57:31,958 : INFO : effective_min_count=1 retains 2203 unique words (100% of original 2203, drops 0)
2020-10-21 07:57:31,959 : INFO : effective_min_count=1 leaves 8970 word corpus (100% of original 8970, drops 0)
2020-10-21 07:57:31,973 : INFO : deleting the raw counts dictionary of 2203 items
2020-10-21 07:57:31,974 : INFO : sample=0.001 downsamples 39 most-common words
2020-10-21 07:57:31,975 : INFO : downsampling leaves estimated 6033 word corpus (67.3% of prior 8970)
2020-10-21 07:57:31,984 : INFO : estimated required memory for 2203 words and 200 dimensions: 4626300 bytes
2020-10-21 07:57:31,985 : INFO : resetting layer weights
2020-10-2

In [70]:
model.wv["セリヌンティウス"]

array([ 9.59435925e-02,  1.32560834e-01,  8.60398188e-02, -1.33975208e-01,
       -9.85954106e-02, -4.55560088e-02, -4.83468175e-02,  1.92997698e-02,
       -9.43999141e-02,  7.80738071e-02, -1.03457170e-02,  3.49839740e-02,
       -1.24563746e-01, -1.22727752e-01, -7.89982527e-02, -3.65503460e-01,
        7.44969845e-02, -1.56704515e-01, -1.46839293e-02,  2.03541573e-02,
       -1.19642645e-01, -8.35007578e-02, -3.01625542e-02, -8.46863389e-02,
       -3.53944413e-02,  6.30898355e-03, -1.97207206e-03, -1.79218203e-01,
        9.75656603e-03,  1.20487534e-01, -1.22297809e-01, -1.86341759e-02,
       -3.90735827e-02, -2.08443888e-02,  9.97724608e-02,  2.47166622e-02,
        6.80463761e-02, -1.99900851e-01, -2.63950825e-02, -1.09833911e-01,
       -7.90279061e-02,  1.01419263e-01, -1.51008904e-01, -2.74323020e-02,
        6.13574833e-02, -5.92333153e-02, -6.80439174e-02, -6.65851757e-02,
        1.16312198e-01,  5.98964281e-03, -1.82370871e-01, -4.73799966e-02,
        8.25392641e-03, -

In [73]:
results = model.most_similar(positive="走る", topn=10)
for result in results:
    print(result[0], '\t', result[1])

と 	 0.9995700120925903
この 	 0.9995616674423218
は 	 0.9995598196983337
に 	 0.9995571374893188
を 	 0.9995555877685547
て 	 0.9995535612106323
た 	 0.9995521306991577
なっ 	 0.9995485544204712
だ 	 0.9995437264442444
こと 	 0.9995386600494385


<ipython-input-73-2705e4d85a64>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  results = model.most_similar(positive="走る", topn=10)
